<a href="https://colab.research.google.com/github/heohyunjun/MusicVAE/blob/main/music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U note-seq==0.0.2 magenta

In [4]:
import magenta
import tensorflow as tf
import tensorflow_datasets as tfds
import pretty_midi
import numpy as np
import mir_eval.display
import librosa.display
import matplotlib.pyplot as plt

# 1. 전처리
1. 4/4 박자가 아닌 파일 삭제
2. 피아노의 88개 음으로 인코딩해서 마디 경계 결정
3. 마디 당 16개음으로 양자화
4. 4-bar(마디) 드럼 패턴인 경우, 4-bar 슬라이딩 윈도우를 사용해서 하나의 flat한 마디 생성
5. 모든 독특한 모노포틱 시퀀스 추출하여 학습 데이터셋 구축 

### 1.1: 4/4박자가 아닌 파일 삭제

In [7]:
import sys
import os
from os import listdir
import string

In [8]:
groove_path = '/content/drive/MyDrive/Colab Notebooks/music/groove'

In [9]:
BEAT = '4-4'

In [10]:
midi_file_path = []

In [11]:
for (root, directories, files) in os.walk(groove_path):
    for file_ in files:
        if file_.endswith('.mid'):
            file_path = os.path.join(root, file_)
            if BEAT not in file_:
                print(file_)
                os.remove(file_path)
            else:
                midi_file_path.append(file_path)

In [12]:
midi_file_path[:5]

['/content/drive/MyDrive/Colab Notebooks/music/groove/drummer4/session1/2_jazz_158_beat_4-4.mid',
 '/content/drive/MyDrive/Colab Notebooks/music/groove/drummer4/session1/1_rock_87_beat_4-4.mid',
 '/content/drive/MyDrive/Colab Notebooks/music/groove/drummer4/session1/9_latin-merengue_158_beat_4-4.mid',
 '/content/drive/MyDrive/Colab Notebooks/music/groove/drummer4/session1/6_blues-shuffle_134_beat_4-4.mid',
 '/content/drive/MyDrive/Colab Notebooks/music/groove/drummer4/session1/7_blues-shuffle_134_beat_4-4.mid']

### 1.2 인코딩된 tempo(박자)를 사용하여 마디 경계를 결정하고, bar(마디) 당 16개의 음(16번 음)으로 양자화

순서
1. midi 데이터 확인
2. 박자 인코딩

- midi 데이터 확인

In [13]:
import mido
midi_ = mido.MidiFile(midi_file_path[0])

In [28]:
midi_.tracks

[<midi track 'Midi Drums' 1559 messages>]


- 4분의 4박자 -> 한마디에 4분표 4개 -> 1비트는 1박자
- ex) 1분에 128박자-> 128bpm

- note_on : 키를 눌러야하는 경우(속도가 0인 경우 해제)
- note_off : 키를 눌르지 않는 경우(속도는 항샹 0)
- channel : 사운드가 전송될 채널, 표준 midi는 동시에 16개 채널 지원
- note : 각 midi 음표 ID에 해당하는 피아노 키보드 키
- velocity : 건반을 두드리는 속도 -> 빠름과 소리는 비례
- time : 마지막 작업 <-> 현재 사작업 사이의 대기 시간
    - 음표 지속 시간 : 동일한 음표에 대한 가장 가까운 2개의 메시지 사이에 있는 각 메시지의 "시간의 합
- track : midi 데이터가 입력되는 트랙

In [105]:
print("파일 이름 :", midi_.filename.split('/')[-1])
print("재생시간 :", midi_.length)
print("mid track : ",midi_.tracks)
print("0번 트랙 :", midi_.tracks[0].name)
print(midi_.charset)
print(midi_.tracks[0].name.encode("latin1")) # 바이트값 출력
print(midi_.tracks[0].name.encode("latin1")[0]) # 바이트값(0~255) 숫자로 출력

파일 이름 : 2_jazz_158_beat_4-4.mid
재생시간 : 49.817268422916676
mid track :  [<midi track 'Midi Drums' 1559 messages>]
0번 트랙 : Midi Drums
latin1
b'Midi Drums'
77


- ticks_per_beat를 이용하면 한 음의 길이를 구할 수 있음

In [14]:
print(midi_.ticks_per_beat)

480


In [36]:
# 트랙 확인
# 6번째까지는 메타 메시지
for m in midi_.tracks[0][:20]:
    print(m)

<meta message track_name name='Midi Drums' time=0>
<meta message instrument_name name='Brooklyn' time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
<meta message smpte_offset frame_rate=24 hours=33 minutes=0 seconds=0 frames=0 sub_frames=0 time=0>
<meta message set_tempo tempo=379747 time=0>
note_on channel=9 note=43 velocity=94 time=25
note_on channel=9 note=38 velocity=87 time=7
note_off channel=9 note=43 velocity=64 time=119
note_off channel=9 note=38 velocity=64 time=8
note_on channel=9 note=38 velocity=105 time=2
note_on channel=9 note=43 velocity=100 time=4
note_off channel=9 note=38 velocity=64 time=124
note_off channel=9 note=43 velocity=64 time=4
note_on channel=9 note=38 velocity=85 time=109
note_on channel=9 note=43 velocity=84 time=10
note_on channel=9 note=43 velocity=90 time=99
note_off channel=9 note=43 velocity=64 time=0
note_off channel=9 note=38 velocity=6

In [16]:
for msg in midi_:
    print("=" *50)
    print(msg)
    print(msg.type)
    

<meta message track_name name='Midi Drums' time=0>
track_name
<meta message instrument_name name='Brooklyn' time=0>
instrument_name
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
time_signature
<meta message key_signature key='C' time=0>
key_signature
<meta message smpte_offset frame_rate=24 hours=33 minutes=0 seconds=0 frames=0 sub_frames=0 time=0>
smpte_offset
<meta message set_tempo tempo=379747 time=0>
set_tempo
note_on channel=9 note=43 velocity=94 time=0.019778489583333333
note_on
note_on channel=9 note=38 velocity=87 time=0.005537977083333333
note_on
note_off channel=9 note=43 velocity=64 time=0.09414561041666666
note_off
note_off channel=9 note=38 velocity=64 time=0.006329116666666666
note_off
note_on channel=9 note=38 velocity=105 time=0.0015822791666666666
note_on
note_on channel=9 note=43 velocity=100 time=0.003164558333333333
note_on
note_off channel=9 note=38 velocity=64 time=0.09810130833333333
note_off
not

In [18]:
# note_on/off 메시지만 추출하는 함수
def get_eventlist(data_file):
    ON = 1
    OFF = 0

    midi = mido.MidiFile(data_file)

    current_time = 0
    eventlist = []
    
    for msg in midi:
#         current_time += msg.time
        current_time = msg.time
        
         # NOTE ON CASE
        if msg.type is 'note_on' and msg.velocity > 0:
            event = [current_time, ON, msg.note, msg.velocity]
            eventlist.append(event)

         # NOTE OFF CASE        
        elif msg.type is 'note_off' or (msg.type is 'note_on' and msg.velocity == 0):
            event = [current_time, OFF, msg.note, msg.velocity]
            eventlist.append(event)
        else:
            pass       
    return eventlist

In [20]:
midi_data = get_eventlist(midi_file_path[0])
for i in midi_data:
    print(i)

[0.019778489583333333, 1, 43, 94]
[0.005537977083333333, 1, 38, 87]
[0.09414561041666666, 0, 43, 64]
[0.006329116666666666, 0, 38, 64]
[0.0015822791666666666, 1, 38, 105]
[0.003164558333333333, 1, 43, 100]
[0.09810130833333333, 0, 38, 64]
[0.003164558333333333, 0, 43, 64]
[0.08623421458333333, 1, 38, 85]
[0.007911395833333333, 1, 43, 84]
[0.07832281875, 1, 43, 90]
[0, 0, 43, 64]
[0.0142405125, 0, 38, 64]
[0.006329116666666666, 1, 38, 75]
[0.0015822791666666666, 0, 43, 64]
[0.09968358749999999, 0, 38, 64]
[0.04509495625, 1, 38, 99]
[0.015822791666666666, 1, 43, 103]
[0.08465193541666666, 0, 38, 64]
[0.015822791666666666, 0, 43, 64]
[0.07278484166666667, 1, 43, 127]
[0.007911395833333333, 1, 38, 127]
[0.09256333124999999, 0, 43, 64]
[0.007911395833333333, 0, 38, 64]
[0.25949378333333334, 1, 38, 127]
[0.019778489583333333, 1, 40, 127]
[0.00237341875, 0, 38, 64]
[0.019778489583333333, 0, 40, 64]
[0.029272164583333333, 1, 36, 73]
[0, 1, 26, 127]
[0.09572788958333332, 0, 36, 64]
[0.005537977

### 1.3 박자 인코딩하기
- 피아노의 건반 수 : 88개
- 인코딩 할 데이터 -> track
    - track에는 음의 높이, 길이 등 객관적인 데이터가 있음, 따라서 트랙정보를 벡터화

In [109]:
# 음표, 속도, 시간 note_on/off 값 린턴
def msg2dict(msg):
    result = dict()
    if 'note_on' in msg:
        on_ = True
    elif 'note_off' in msg:
        on_ = False
    else:
        on_ = None
    result['time'] = int(msg[msg.rfind('time'):].split(' ')[0].split('=')[1].translate(
        str.maketrans({a: None for a in string.punctuation})))
    
    if on_ is not None:
        for k in ['note', 'velocity']:
            result[k] = int(msg[msg.rfind(k):].split(' ')[0].split('=')[1].translate(
                str.maketrans({a: None for a in string.punctuation})))
    print(result)
    return [result, on_]

In [110]:
def switch_note(last_state, note, velocity, on_=True):
    # 피아노는 88건반 
    result = [0] * 88 if last_state is None else last_state.copy()
    # 범위가 왜 이렇게 되는가?
    if 21 <= note <= 108:
        result[note-21] = velocity if on_ else 0
    return result

In [111]:
# 각 트랙의 메시지 전처리 후, 리턴
def get_new_state(new_msg, last_state):
    new_msg, on_ = msg2dict(str(new_msg))
    new_state = switch_note(last_state, note=new_msg['note'], velocity=new_msg['velocity'], on_=on_) if on_ is not None else last_state
    return [new_state, new_msg['time']]


In [112]:
# 트랙의 메시지 변환
def track2seq(track):
    result = []

    # 0번쨰는 <midi track 'Midi Drums' 1559 messages>
    # 0번쨰는 파일 설명, 박자 서명, 조표, 템포 등과 같은 정보를 저장하는 메타 메시지
    last_state, last_time = get_new_state(str(track[0]), [0]*88)
    for i in range(1, len(track)):
        new_state, new_time = get_new_state(track[i], last_state)
        if new_time > 0:
            result += [last_state]*new_time
        last_state, last_time = new_state, new_time
    return result

In [80]:
len(midi_.tracks[0])

1559

In [107]:
midi_.tracks[0][7]

<message note_on channel=9 note=38 velocity=87 time=7>

In [40]:
def mid2arry(mid, min_msg_pct=0.1):
    # 파일별 트랙길이 리스트
    tracks_len = [len(tr) for tr in mid.tracks]
    
    # 최소 트랙 길이
    min_n_msg = max(tracks_len) * min_msg_pct
    
    # convert each track to nested list
    all_arys = []
    
    # 트랙 처리
    for i in range(len(mid.tracks)):
        # 트랙 길이가 최소 트랙 길이보다 길면
        if len(mid.tracks[i]) > min_n_msg:

            # track to sequence -> 한 트랙의 메시지 하나씩 처리
            ary_i = track2seq(mid.tracks[i])
            all_arys.append(ary_i)

    
    max_len = max([len(ary) for ary in all_arys])
    for i in range(len(all_arys)):
        if len(all_arys[i]) < max_len:
            all_arys[i] += [[0] * 88] * (max_len - len(all_arys[i]))
    all_arys = np.array(all_arys)
    all_arys = all_arys.max(axis=0)
    
    sums = all_arys.sum(axis=1)
    ends = np.where(sums > 0)[0]
    return all_arys[min(ends): max(ends)]

In [49]:
result_array = mid2arry(midi_)

In [115]:
result_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [116]:
result_array.shape


(62943, 88)